# Segmenting and Clustering Neighborhoods in the City of Toronto

### -------------------------------Answer:1-------------------------------------------------------

In [ ]:
#install Beautifulsoup
!conda install -c conda-forge beautifulsoup4 --yes

In [ ]:
#install lxml
!conda install -c conda-forge lxml --yes

In [1]:
#import libraries
from bs4 import BeautifulSoup
import requests

In [2]:
#Access the wiki page text using Beautifulsoup

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

source = requests.get(url).text

soup=BeautifulSoup(source,'lxml')
    
#print(soup.prettify())    

 
    

In [3]:
#Retreive table header  from <th> tag  (i.e. column names - Postcode,Borough,Neighbourhood)

columns=[]

table= soup.find('table',class_='wikitable sortable')

table_header = table.find_all('th')

columns=[col.text.strip() for col in table_header]

#print(columns)


#Retreive table data from each row , each cell  ( from <tr> <td>  tags)

row_data=[]

table_rows=table.find_all('tr')

for tr in table_rows:
    if(tr.find('td')):
        td=tr.find_all('td')
        row_data.append([i.text.strip() for i in td])

#print(row_data)        
        
 

In [4]:
import pandas as pd

#convert table data in to dataframe

neighbourhood=pd.DataFrame(row_data)

neighbourhood.columns=['PostalCode','Borough','Neighbourhood']


#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
neighbourhood=neighbourhood[neighbourhood['Borough'] != 'Not assigned']


#More than one neighborhood can exist in one postal code area. For example, in the table on the 
#Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
#separated with a comma as shown in row 11 in the above table.

neighbourhood_combined=neighbourhood.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ','.join(x)).reset_index()

neighbourhood_combined.head(10)

#Please note that the order of postalcode in the below output may not be same as given in the question, but 
#the data in each row are correct.

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be 
#the same as the borough. So for the 9th cell in the table on the Wikipedia page,
#the value of the Borough and the Neighborhood columns will be Queen's Park.

neighbourhood_combined.loc[neighbourhood_combined["Neighbourhood"] == "Not assigned","Neighbourhood"]=neighbourhood_combined["Borough"]

neighbourhood_combined.head(10)

#Please note that the order of postalcode in the below output may not be same as given in the question, but 
#the data in each row are correct.

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [6]:
neighbourhood_combined.shape

(103, 3)